In [2]:
# required imports
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from grid_parser import *
from unit_parser import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *
from executor import *
from dfs import *
from dfs_viewer import *
from main import *
from melee import *
from mech import *
from vek import *


In [3]:
# example setup of the board
# the gridparser string specifies the type of environment tiles the battleground consists of
# eg. 2w2g=m2w/  means row has 2 water tiles followed by 2 grass, a double power (=), a mountain, 2 water, and (/) which denotes next row 

# the unitparser string specifies the veks and meks that are on the board
# 8o/ means 8 empty spaces then next row
# 5od/ means 5 empty followed by deathball (volcano bomb that does 5 damage)
# m means mech (followed by mech name, max health, health, and moves)
# v means vek (followed by vek name, max health, health, attack order, direction it's facing (optional), and offset (optional))

# check main.py for examples on how to setup board without the unit/grid parser
def renfield1():
    grid = GridParser.from_string("2w2g=m2w/gw=4gw/7gm/7g=/wg=5g/=5gmg/4g=g=w/mw5gw")
    return UnitParser(grid).from_string("""
    8o/
    5od/
    3om(Artillery Mech_5_5_4)m(Siege Mech_5_5_3)/
    odom(Boulder Mech_5_5_4)m(Renfield Bomb_4_4_0)v(Firefly_3_3_#4_E)/
    5ov(Blobber_3_3_#5)d/
    ov(Hornet_2_2_#3_W)d2os/
    ossv(Beetle Leader_6_6_#2_E)ov(Blob_1_1_#6_N)/
    """)

In [4]:
def renfield2():
    grid = GridParser.from_string("2w2g=m2w/gw=4gw/7gm/7g=/wg=5g/=5gmg/4g=g=w/mw5gw")
    return UnitParser(grid).from_string("""
    8o/
    ov(Alpha Hornet_4_4_#5_E)od-m(Artillery Mech_5_5_4)/
    6od-m(Siege Mech_5_5_3)/
    d-m(Boulder Mech_5_5_4)3om(Renfield Bomb_4_4_0)/
    7os/
    2ov(Hornet_2_2_#4_N)fv(Hornet_2_2_#7_S)s/
    3ov(Beetle Leader_6_6_#3_N)ov(Psion Tyrant_2_2_#6)
    """)

In [5]:
def renfield3():
    grid = GridParser.from_string("2w2g=m2w/gw=4gw/7gm/7g=/wg=5g/=5gmg/4g=g=w/mw5gw")
    return UnitParser(grid).from_string("""
    8o/
    4om(Siege Mech_5_5_3)om(Artillery Mech_5_5_4)/
    8o/
    2o2dm(Renfield Bomb_4_4_0)ov(Firefly_3_3_#4_N)/
    3o2d-u(Boulder)2os/
    ov(Beetle Leader_6_4_#3_S)sd-u(Boulder)v(Alpha Digger_4_4_#5_N)u(Boulder)/
    om(Boulder Mech_5_5_4)os/
    """)

In [6]:
def renfield4():
    grid = GridParser.from_string("2w2g=m2w/gw=w3gw/6gwm/w6g=/wg=/=/4g=g=w/mw5gw")
    return UnitParser(grid).from_string("""
    5od/
    5od-m(Artillery Mech_5_5_4)/
    5od/
    ov(Alpha Scarab_4_4_#4_E_3)2om(Renfield Bomb_4_4_0)d/
    ov(Beetle Leader_6_3_#3_E)3od-m(Siege Mech_5_4_3)v(Alpha Scarab_4_4_#6_S_2)/
    os-fss-fv(Beetle_4_4_#5_N)d/
    of3od/
    3om(Boulder Mech_5_5_4)od
     """)

In [7]:
def acid1():
    grid = GridParser.from_string("5gf2g/g-g2wg-f/g-2gwg-g/f3gf3g/2g=-4g/5g=gf/wf2g3fg/3w5g")
    return UnitParser(grid).from_string("""
    8o/
    8o/
    8o/
    2om(Leap Mech_3_3_4)m(Unstable Mech_3_3_3)m(Nano Mech_2_2_4)/
    4ov(Scarab_2_2_#2_W_2)v(Scarab_2_2_#3_W_2)/
    3ov(Firefly_3_3_#1_N)/
    3os/
    4os
     """)
    return grid

In [41]:
# create the a "grid" which can be viewed with grid.show()
grid = renfield1()

In [26]:
# calculate all potential moves up to a max of (7) moves into the future (use a smaller # if taking too long)
dfs = DFS(grid, 7)

In [27]:
# visualization of grid setup with grid/unit parser
# shows water tiles, grass tiles, mountain tiles, units w/health, and power centers
grid.show()

,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,🟢,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,Artillery Mech ♡♡♡♡♡,Siege Mech ♡♡♡♡♡,🟢,🟢,🗻
3,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,Firefly ♡♡♡,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,Blobber ♡♡♡,🟢,🟢
5,🏘️ ϟϟ,Hornet ♡♡,🟢,🟢,🟢,🟢,🗻,🟢
6,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🏘️ ϟϟ,Blob ♡,🏘️ ϟϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


In [28]:
# this end commands list shows you the order of the vek's moves (and environment) that will occur when the round ends
grid.end_commands

[DAMAGE (1, 5) -5,
 DAMAGE (3, 1) -5,
 DAMAGE (4, 6) -5,
 DAMAGE (5, 2) -5,
 VekCharge at (6, 3) charging EAST,
 DAMAGE (5, 0) -1,
 VekBeam at (3, 5) heading EAST,
 [[DAMAGE (5, 5) -1, DAMAGE (7, 5) -1, DAMAGE (6, 6) -1, DAMAGE (6, 4) -1], DAMAGE (6, 5) -1],
 DAMAGE (5, 5) -1,
 DAMAGE (6, 1) -1,
 DAMAGE (6, 2) -1]

In [30]:
# uses the depth first search (dfs) object and the grid layout to create a visualization of all potential outcomes
# for the given grid
v = DFSViewer(renfield1(), dfs)

In [34]:
# provides a summary of the top 100 outcomes (doesn't show duplicate outcomes)
# the summary includes the index (0-99th best outcome), the total power remaining, the # of veks remaining,
# the total vek health, the # of mechs remaining, and the total mech health
v.quick_peek(100)

Power	Veks	Vek Total Health	Mechs	Mech Total Health
0) [12, 1, 6, 4, 19]
8) [12, 2, 4, 4, 19]
9) [11, 1, 4, 4, 19]
69) [12, 2, 6, 4, 19]
72) [11, 1, 6, 4, 19]
88) [10, 1, 2, 4, 19]
89) [12, 2, 7, 4, 19]
98) [12, 2, 8, 4, 19]


In [50]:
# prints the 0th best outcome results (show_grid =True will show the resulting grid of the outcome)
# (verbose = True) will show the order in which to move your pieces in order to end up with the selected outcome
v.show(0, show_grid = True, verbose = True)

,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,🟢,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,Artillery Mech ♡♡♡♡♡,Siege Mech ♡♡♡♡♡,🟢,🟢,🗻
3,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,Firefly ♡♡♡,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,Blobber ♡♡♡,🟢,🟢
5,🏘️ ϟϟ,Hornet ♡♡,🟢,🟢,🟢,🟢,🗻,🟢
6,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🏘️ ϟϟ,Blob ♡,🏘️ ϟϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


MOVE (2, 3) to (1, 3)


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🗻
3,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,Firefly ♡♡♡,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,Blobber ♡♡♡,🟢,🟢
5,🏘️ ϟϟ,Hornet ♡♡,🟢,🟢,🟢,🟢,🗻,🟢
6,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🏘️ ϟϟ,Blob ♡,🏘️ ϟϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


REGULAR SHELL at (7, 3)


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🗻
3,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,Firefly ♡♡♡,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,Blobber ♡♡♡,🟢,🟢
5,🏘️ ϟϟ,Hornet ♡♡,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟϟ,Blob ♡,🏘️ ϟϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


...

BOULDER SHELL at (3, 5)


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢,🗻
3,Boulder Mech ♡♡♡♡♡,🟢,🟢,🟢,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟϟ,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,Blobber ♡♡♡,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟϟ,Blob ♡,🏘️ ϟϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


215


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢,🗻
3,Boulder Mech ♡♡♡♡♡,🟢,🟢,🟢,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 12, 'Veks': 1, 'Vek Total Health': 6, 'Mechs': 4, 'Mech Total Health': 19}


In [51]:
# once again just shows the final grid outcome
v.show(0, show_grid = True)

215


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢,🗻
3,Boulder Mech ♡♡♡♡♡,🟢,🟢,🟢,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 12, 'Veks': 1, 'Vek Total Health': 6, 'Mechs': 4, 'Mech Total Health': 19}


In [37]:
# shows you how the top 20 outcomes look, 
# good for deciding how you want the game to go since the best outcome isn't always obvious
for i in range(20):
    v.show(i)

215


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢,🗻
3,Boulder Mech ♡♡♡♡♡,🟢,🟢,🟢,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 12, 'Veks': 1, 'Vek Total Health': 6, 'Mechs': 4, 'Mech Total Health': 19}
215


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,🟢,🗻
3,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 12, 'Veks': 1, 'Vek Total Health': 6, 'Mechs': 4, 'Mech Total Health': 19}
215


...

{'Power': 11, 'Veks': 1, 'Vek Total Health': 4, 'Mechs': 4, 'Mech Total Health': 19}
209


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,🟢,🟢,🟢,🟢,🗻
3,Artillery Mech ♡♡♡♡♡,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 11, 'Veks': 1, 'Vek Total Health': 4, 'Mechs': 4, 'Mech Total Health': 19}
209


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,🟢,🟢,🟢,🟢,🗻
3,Artillery Mech ♡♡♡♡♡,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 11, 'Veks': 1, 'Vek Total Health': 4, 'Mechs': 4, 'Mech Total Health': 19}


In [38]:
v.show(20)

209


,0,1,2,3,4,5,6,7
0,💦,💦,🟢,🟢,🏘️ ϟϟ,🗻,💦,💦
1,🟢,💦,🏘️ ϟϟ,Siege Mech ♡♡♡♡♡,🟢,🟢,🟢,💦
2,🟢,🟢,🟢,🟢,🟢,🟢,🟢,🗻
3,Artillery Mech ♡♡♡♡♡,🟢,🟢,Boulder Mech ♡♡♡♡♡,Renfield Bomb ♡♡♡♡,🟢,🟢,🏘️ ϟϟ
4,💦,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🟢
5,🏘️ ϟ,🟢,🟢,🟢,Beetle Leader ♡♡♡♡,🟢,🗻,🟢
6,🟢,🟢,🟢,🟢,🏘️ ϟ,🟢,🏘️ ϟ,💦
7,🗻,💦,🟢,🟢,🟢,🟢,🟢,💦


{'Power': 11, 'Veks': 1, 'Vek Total Health': 4, 'Mechs': 4, 'Mech Total Health': 19}


In [ ]:
# not sure what the rest of this notebook does but hopefully these above comments helps you out sarcus
# will be happy if you are actually able to end up using this
# you may need to implement different unique mechs / veks / weapons as you encounter them
# if you do feel free to send a Pull Request, would be neat

In [ ]:
# be able to call the right functions after parsing
# make sure those functions do what theyre supposed to


In [ ]:
grid = reset_grid4()

In [ ]:
siege_mech = Mech("Siege Mech", 5, 5, 3)

In [ ]:
siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))

In [ ]:
siege_mech.gen_actions()

In [ ]:
stop

In [ ]:
# tests to pass to make it work

# first is make the gridbuilder grid work

def test_grid_builder_places_correct_tiles_from_string():
    test_string = "2w2g=m2w/gw=4gw/6gm/6g=/wg=5g/=5gmg/4g=g=wmw5gw"
    grid = GridParser.from_string(test_string)
    df = grid.show()
    
    def check_tile_at(coord, string):
        x,y = coord
        tile_repr = repr(df.iloc[x][y])
        assert tile_repr == string, f"{tile_repr} != {string}"
        
    
    def matches(string):
        return string
    
    check_tile_at((0,0), matches('💦'))
    check_tile_at((0,1), matches('💦'))
    check_tile_at((0,4), matches('🏘️ ϟϟ'))
    check_tile_at((0,5), matches('🗻'))
    check_tile_at((0,6), matches('💦'))
    check_tile_at((0,7), matches('💦'))
    check_tile_at((7,7), matches('💦'))


test_grid_builder_places_correct_tiles_from_string()

In [ ]:
blobber = Stuff(5, "hello", "goodbye").add_move("MOVES")

In [ ]:
blobber.execute()

In [ ]:
# 0 ground tile
# 1 water
# 2 mounatin
# 3 civilian 1
# 33 civilian 2

In [ ]:
# chessable 8x8 button grid
# displays grid from string


In [ ]:
def reset_grid5():
    [1,1,0,0,33,2,1,1]
    [0,1,33]
    
    
    
    

In [ ]:
# firefly = Vek("Firefly", 4, 2, 2)
# alpha_firefly = Vek("Alpha Firefly", 5, 5, 2)

# hornet = Vek("Hornet", 2, 2, 5, True)
# alpha_hornet = Vek("Alpha Hornet", 4, 4, 5, True)

# scarab = Vek("Scarab", 2, 2)
# alpha_scarab = Vek("Alpha Scarab", 4, 4)

# beetle = Vek("Beetle", 4, 4, 2)

# blobber = Vek("Blobber", 3, 3, 2)
# blob = Unit("Blob", 1, 1, 0)

# psy = Vek("Psion Tyrant", 2, 2, 2, True)

In [ ]:
# MOV (7, 3) to (5, 5)
# MOVE (3, 2) to (3, 4)
# BOULDER SHELL at (5, 1)
# MOVE (5, 3) to (7, 3)
# REGULAR SHELL at (7, 1)
# CLUSTER SHELL at (3, 2)

In [ ]:
# drag and drop buttons
# map creator
# 2d map creator

In [ ]:
# Game(reset_grid(), True)

In [ ]:
# grid = reset_grid3()

In [ ]:
# grid.end_commands[4]

In [ ]:
# Game(reset_grid2(), True)

In [ ]:
grid = reset_grid4()
display(grid.show())
dfs = DFS(grid, 8)

In [ ]:
v = DFSViewer(reset_grid4(), dfs)

In [ ]:
v.show(0, True)

In [ ]:
v.ls[i][1]

In [ ]:
explored = set()
for i in range(100):
    if v.ls[i][1] not in explored:
        v.show(i)
        explored.add(v.ls[i][1])

In [ ]:
v.show(2, True)

In [ ]:
v.show(0, True)

In [ ]:
grid = reset_grid3()





In [ ]:
history = {}
for string, action in moves:
    name = repr(action)
    if string == "DO" and name in history:
        print(f"NOPE {name}")
    if string == "DO":
        history[name] = True

In [ ]:
sdf = {}

In [ ]:
pip = ["Stuff", "Thing", sdf]

In [ ]:
del sdf

In [ ]:
pip

In [ ]:
pip.pop(2)

In [ ]:
pip


In [ ]:
dfs = DFS(grid, 6)
moves = dfs.protected[0]

def try_moves(moves):
    grid = reset_grid3()
    history = {}
    frontier = DFSViewer.gen_frontier(grid)
    for string, action in moves:
        name = repr(action)
        if string == "DO":
            com = frontier[name]
            com.execute()
            score = DFS.rate(grid)
            history[id(action)] = (com, score)
        else:
            com, score = history[id(action)]
            com.undo()
            new_score = DFS.rate(grid)
            if new_score != score:
                print(f"{new_score} does not equal {score} after undoing {command} with id {id(action)}")
                break

        frontier = DFSViewer.gen_frontier(grid)

In [ ]:
history[2132117014792]

In [ ]:
id(grid)

In [ ]:
grid = reset_grid3()
v = DFSViewer(grid, dfs)

In [ ]:
v.show(1, True)

In [ ]:
stop

In [ ]:
# grid.show()

In [ ]:
# MOVE (3, 2) to (1, 2)
# CLUSTER SHELL at (6, 2)
# MOVE (5, 3) to (6, 1)
# REGULAR SHELL at (6, 6)
# MOVE (7, 3) to (7, 2)
# BOULDER SHELL at (4, 2)

In [ ]:
grid = reset_grid()
DFSViewer(grid, dfs)

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
stop

In [ ]:
DFS.rate(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
stop

In [ ]:
grid = reset_grid()

In [ ]:
grid.show()

In [ ]:
DFS.rate_dict(grid)

In [ ]:
dfs.show()

In [ ]:
dfs.explored

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
sm.ge

In [ ]:
choice.command.__dict__.copy()

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
ex = Executor()

In [ ]:
ex.execute(siege_mech.gen_actions()[2])

In [ ]:
grid.show()

In [ ]:
ex.execute(siege_mech.gen_actions()[4])

In [ ]:
grid.show()

In [ ]:
ex

In [ ]:
ex.undo()

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
ex = Executor()

score = DFS.rate(grid)

DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[2][0])
score = DFS.rate(grid)

show = grid.show()

dic = DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
score = DFS.rate(grid)

frontier.sort(key=lambda x:x[1], reverse=True)

score = DFS.rate(grid)

ex.execute(frontier[0][0])
score = DFS.rate(grid)

dic = DFS.rate_dict(grid, True)

show = grid.show()

In [ ]:
print(score)

In [ ]:
ex.undo()
score = DFS.rate(grid)

show = grid.show()

ex.execute(frontier[1][0])
score = DFS.rate(grid)

grid.show()

ex.undo()
score = DFS.rate(grid)

grid.show()

ex.execute(frontier[2][0])
score = DFS.rate(grid)

grid.show()

grid.get_tile((3,6)).visitor

grid.get_tile((5,5)).visitor

grid.get_tile((4,6)).type_object.health

grid.get_tile((4,5)).type_object.health

DFS.rate_dict(grid, True)

grid.show()

In [ ]:
print(score)

new score 42 != old score 37 at 
MOVE (7, 3) to (5, 5)
MOVE (3, 2) to (3, 4)
CLUSTER SHELL at (3, 2)
BOULDER SHELL at (5, 1)
MOVE (5, 3) to (3, 3)
HEAL +1

[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, 'UNDO', REGULAR SHELL at (7, 1), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 2), HEAL +1, 'UNDO', REGULAR SHELL at (6, 2), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 3), REGULAR SHELL at (3, 6), 'UNDO', REGULAR SHELL at (3, 7), 'UNDO', HEAL +1]

In [ ]:
stop

In [ ]:
stop

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
grid.show()

In [ ]:
frontier = DFS.gen_rated_frontier(grid)

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    

In [ ]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

In [ ]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))